In [1]:
import os
import re
import json
import warnings
import huggingface_hub

from openai import OpenAI

os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings("ignore", category=FutureWarning)

from dotenv import load_dotenv
load_dotenv("../keys.env")

upstage_api_key = os.getenv("UPSTAGE_API_KEY")
os.environ['UPSTAGE_API_KEY'] = upstage_api_key

openai_api_key = os.getenv('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = openai_api_key

hf_token = os.getenv("HF_TOKEN")
huggingface_hub.login(hf_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/pervinco/.cache/huggingface/token
Login successful


In [2]:
client = OpenAI()
model = "gpt-4o"

In [3]:
def load_jsonl(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return [json.loads(line) for line in f]

In [4]:
def clean_json_response(response):
    # 코드 블록(예: ```json, ```) 제거
    cleaned_response = re.sub(r'```(?:json)?', '', response).strip()
    
    return cleaned_response

In [20]:
def create_standalone_query(dialogue, model, client, src_lang):
    standalone_content = (
        "입력된 내용이 한 줄의 문장인지 여러 줄의 대화 내용인지 분류하세요. "
        "반드시 JSON 형식으로 응답하며, 키와 값은 모두 이중 따옴표로 감쌉니다."
        "- 단일 문장인 경우 : {\"multi_turn\": false, \"query\": \"입력 문장\"} "
        "- 여러 줄의 대화 내용인 경우 : {\"multi_turn\": true, \"query\": \"대화 내용을 종합하여 만든 새로운 질문\"}"
    )
    
    # LLM 호출
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": standalone_content},
            {"role": "user", "content": dialogue}
        ],
    )
    
    response = completion.choices[0].message.content
    cleaned_response = clean_json_response(response)
    
    try:
        json_response = json.loads(cleaned_response)
    except json.JSONDecodeError:
        return {"error": "Invalid JSON response", "response": response}
    
    return json_response

In [21]:
def load_query(query_path, model, client, output_path):
    queries = load_jsonl(query_path)
    expanded_queries = []

    for query_data in queries:
        eval_id = query_data['eval_id']
        msg_history = query_data['msg']
        
        conversation_history = " ".join([msg['content'] for msg in msg_history if msg['role'] == 'user'])
        
        result = create_standalone_query(conversation_history, model, client, src_lang="ko")
        print(eval_id)
        print(conversation_history)
        print(result['query'], '\n')
        
        expanded_queries.append({
            "eval_id": eval_id,
            "query": result,
        })
    
    # Save expanded queries to a jsonl file
    with open(output_path, 'w', encoding='utf-8') as f:
        for query in expanded_queries:
            json.dump(query, f, ensure_ascii=False)
            f.write('\n')

In [22]:
load_query("../dataset/eval.jsonl", model, client, "../dataset/processed_query.jsonl")

78
나무의 분류에 대해 조사해 보기 위한 방법은?
나무의 분류에 대해 조사해 보기 위한 방법은? 

213
각 나라에서의 공교육 지출 현황에 대해 알려줘.
각 나라에서의 공교육 지출 현황에 대해 알려줘. 

107
기억 상실증 걸리면 너무 무섭겠다. 어떤 원인 때문에 발생하는지 궁금해.
기억 상실증의 원인은 무엇인가요? 

81
통학 버스의 가치에 대해 말해줘.
통학 버스의 가치에 대해 말해줘. 

280
Dmitri Ivanovsky가 누구야?
Dmitri Ivanovsky가 누구야? 

10
피임을 하기 위한 방법중 약으로 처리하는 방법은 쓸만한가?
피임을 하기 위한 방법중 약으로 처리하는 방법은 쓸만한가? 

100
헬륨이 다른 원소들과 반응을 잘 안하는 이유는?
헬륨이 다른 원소들과 반응을 잘 안하는 이유는? 

279
문맹 비율이 사회 발전에 미치는 영향은?
문맹 비율이 사회 발전에 미치는 영향은? 

42
이란 콘트라 사건이 뭐야 이 사건이 미국 정치에 미친 영향은?
이란 콘트라 사건이 무엇인지와 이 사건이 미국 정치에 미친 영향을 설명해 주세요. 

308
자기장이 얼마나 센지 표현하는 방식은?
자기장이 얼마나 센지 표현하는 방식은? 

205
피를 맑게 하고 몸 속의 노폐물을 없애는 역할을 하는 기관은?
피를 맑게 하고 몸 속의 노폐물을 없애는 역할을 하는 기관은? 

289
글리코겐의 분해는 인체에서 왜 필요한가?
글리코겐의 분해는 인체에서 왜 필요한가? 

268
빗방울이 점점 커지게 되는 요인은?
빗방울이 점점 커지게 되는 요인은? 

18
기체의 부피나 형태가 왜 일정하지 않을까?
기체의 부피나 형태가 왜 일정하지 않을까? 

9
식물이 빛을 에너지로 변환하는 과정에 대해 설명해줘.
식물이 빛을 에너지로 변환하는 과정에 대해 설명해줘. 

101
직류와 교류 전류의 차이에 대해 알려줘.
직류와 교류 전류의 차이에 대해 알려줘. 

236
기름과 물이 섞여 있을 수 있나?
기름과 물이 섞여 있을 수 있나? 

59
인간이 2세를 생산할 때

In [9]:
def query_expansion(query, model:str, client:OpenAI):
    content = (
        """
        당신은 한국어 질의를 이해하고 개선하여 검색 시스템이 정확한 문서를 찾을 수 있도록 돕는 전문가입니다.

        **당신의 임무:**

        1. **질의 분석**: 사용자의 질의를 면밀히 분석하여 의도를 깊이 이해합니다. 이때 체인 오브 소트(chain-of-thought) 방식으로 내부적으로 추론하지만, 최종 답변에는 포함하지 않습니다.

        2. **의도 유지 및 확장**: 사용자의 원래 의도를 유지하면서, 검색 시스템이 관련 문서를 더 잘 찾을 수 있도록 질의를 명확하고 상세하게 확장합니다.

        3. **다의어 처리**: 다의어가 포함된 경우, 문맥에서 가장 적절한 의미를 선택하여 질의를 명확히 합니다.

        4. **분야 명확화**: 키워드가 여러 분야에서 사용될 수 있다면, 사용자가 의도한 분야나 가장 가능성이 높은 분야를 명확히 합니다.

        5. **모호성 해소**: 질의가 불명확하거나 모호하다면, 일반적인 해석을 추가하여 검색의 정확도를 높입니다.

        **지침:**

        - 최종 답변에는 **분석이나 추론 과정**을 포함하지 말고, **확장된 자연스러운 질의만을 제공합니다**.
        - 반환 형식은 반드시 **JSON 포맷**이며, 모든 문자열은 쌍따옴표로 감싸야 합니다.
        - 형식 예시: `{ "query": "확장된 자연스러운 질의" }`

        **예시:**

        예제 입력:
        ```
        Thomas Alva Edison은 누구인가요?
        ```
        확장된 질의:
        ```json
        { "query": "토머스 알바 에디슨(Thomas Alva Edison)은 어떤 인물인가요?" }
        ```

        예제 입력:
        ```
        온난 전선이 발생하면 이후 날씨는 어떻게 되나?
        ```
        확장된 질의:
        ```json
        { "query": "기상학에서 온난 전선이 발생한 후의 날씨 변화와 기상 현상은 어떻게 되나요?" }
        ```

        예제 입력:
        ```
        난관의 기능에 대해 알려줘.
        ```
        확장된 질의:
        ```json
        { "query": "인체 생식 기관 중 난관의 역할과 기능은 무엇인가요?" }
        ```
        """
    )

    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role" : "system", "content" : content},
            {"role" : "user", "content" : query}
            ],
    )
        
    response = completion.choices[0].message.content
    response = clean_json_response(response)
        
    try:
        json_response = json.loads(response)
    except json.JSONDecodeError:
        return {"error": "Invalid JSON response", "response": response}
        
    return json_response


In [10]:
def load_jsonl(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return [json.loads(line) for line in f]

def load_query(query_path, model, client, output_path):
    queries = load_jsonl(query_path)
    expanded_queries = []

    for query_data in queries:
        eval_id = query_data['eval_id']
        msg_history = query_data['msg']
        
        conversation_history = " ".join([msg['content'] for msg in msg_history if msg['role'] == 'user'])
        
        expanded_query_result = query_expansion(conversation_history, model, client)
        print(eval_id)
        print(conversation_history)
        print(expanded_query_result['query'], '\n')
        
        expanded_queries.append({
            "eval_id": eval_id,
            "original_conversation": conversation_history,
            "expanded_query": expanded_query_result.get('query', 'Expansion failed')
        })
    
    # Save expanded queries to a jsonl file
    with open(output_path, 'w', encoding='utf-8') as f:
        for query in expanded_queries:
            json.dump(query, f, ensure_ascii=False)
            f.write('\n')

In [11]:
load_query("../dataset/eval.jsonl", model, client, "../dataset/expanded_query_v2.jsonl")

78
나무의 분류에 대해 조사해 보기 위한 방법은?
식물학 및 생물학에서 나무의 분류 체계와 분류 방법을 조사하는 방법은 무엇인가요? 

213
각 나라에서의 공교육 지출 현황에 대해 알려줘.
세계 각 나라의 공교육 분야 예산 지출 현황과 각국 정부가 교육에 투자하는 비율은 어떻게 되나요? 

107
기억 상실증 걸리면 너무 무섭겠다. 어떤 원인 때문에 발생하는지 궁금해.
기억 상실증의 원인과 발생 메커니즘은 무엇인가요? 

81
통학 버스의 가치에 대해 말해줘.
통학 버스가 학생들에게 제공하는 장점과 가치는 무엇인가요? 교통 안전성, 비용 절감, 시간 효율성 측면에서 설명해주세요. 

280
Dmitri Ivanovsky가 누구야?
과학자 드미트리 이바노프스키(Dmitri Ivanovsky)는 어떤 인물이고, 그의 주요 업적은 무엇인가요? 

10
피임을 하기 위한 방법중 약으로 처리하는 방법은 쓸만한가?
피임을 위해 약물(피임약)을 사용하는 방법의 효과성과 장단점에 대해서 알려주세요. 

100
헬륨이 다른 원소들과 반응을 잘 안하는 이유는?
헬륨 원소가 화학적으로 다른 원소들과 반응을 잘 하지 않는 이유는 무엇인가요? 

279
문맹 비율이 사회 발전에 미치는 영향은?
문맹 비율이 사회 발전과 경제, 교육, 사회적 측면에 미치는 영향은 무엇인가요? 

42
이란 콘트라 사건이 뭐야 이 사건이 미국 정치에 미친 영향은?
이란 콘트라 사건의 개요와 이 사건이 미국 정치에 미친 영향은 무엇인가요? 

308
자기장이 얼마나 센지 표현하는 방식은?
물리학에서 자기장의 세기를 표현하는 방식에는 어떤 것들이 있나요? 

205
피를 맑게 하고 몸 속의 노폐물을 없애는 역할을 하는 기관은?
인체에서 피를 맑게 하고 몸 속의 노폐물을 제거하는 역할을 하는 기관은 무엇인가요? 

289
글리코겐의 분해는 인체에서 왜 필요한가?
인체에서 글리코겐의 분해가 필요한 이유와 그 과정에서 발생하는 생리적 역할은 무엇인가요? 

268
빗방울이 점점 커지게 되는 요인은?
기상학에서

In [13]:
def query_expansion_with_chain_of_thought(query, model: str, client):
    # 전체 대화를 멀티턴으로 구성
    messages = [
        {"role": "system", "content": "당신은 검색 쿼리 확장 전문가입니다. 주어진 질문을 분석하여 더 나은 검색 결과를 위해 간결하고 명확한 쿼리를 만드세요. 세부 정보는 필요할 때만 추가하고, 불필요한 정보는 넣지 마세요."},
        
        # Step 1: 의도 파악
        {"role": "user", "content": f"다음 질문의 의도를 간결하게 파악하세요: {query}"},
        {"role": "assistant", "content": "질문의 의도는 다음과 같습니다."},
        
        # Step 2: 핵심 단어 파악
        {"role": "user", "content": f"이제 질문에서 파악한 의도를 바탕으로, 질문의 핵심 단어만 추출하세요. 질문: {query}"},
        {"role": "assistant", "content": "질문의 핵심 단어는 다음과 같습니다."},
        
        # Step 3: 새로운 쿼리 생성
        {"role": "user", "content": f"파악한 의도와 핵심 단어를 바탕으로, 질문을 간결하고 명확하게 다시 작성하세요. 그리고 따옴표는 사용하지 마세요. 질문: {query}"},
        {"role": "assistant", "content": "확장된 질문은 다음과 같습니다."}
    ]
    
    # 멀티턴 대화를 한번에 처리
    completion = client.chat.completions.create(
        model=model,
        messages=messages,
    )
    
    # 응답 내용 가져오기
    response = completion.choices[0].message.content
    
    # 결과를 반환
    return response.strip()


In [15]:
queries = load_jsonl("../dataset/processed_eval.jsonl")
output_path = "../dataset/expanded_query_v3.jsonl"

with open(output_path, 'w', encoding='utf-8') as f:
    expanded_queries = []
    
    for query_data in queries[:10]:  # 첫 10개 쿼리에 대해서만 실행
        eval_id = query_data['eval_id']
        query = query_data['query']
        
        # 쿼리 확장 수행
        result = query_expansion_with_chain_of_thought(query, model, client)
        print(eval_id)
        print(query)
        print(result, '\n')
        
        # 확장된 쿼리 리스트에 추가
        expanded_queries.append({
            "eval_id": eval_id,
            "query": result
        })
        
        json.dump({
            "eval_id": eval_id,
            "query": result
        }, f, ensure_ascii=False)
        f.write('\n')

78
나무의 분류에 대해 조사해 보기 위한 방법은?
나무 분류 방법 조사 

213
각 나라에서의 공교육 지출 현황에 대해 알려줘.
세계 각국의 공교육 지출 현황 

107
기억 상실증의 원인은 무엇인가요?
기억 상실증 원인 

81
통학 버스의 가치에 대해 말해줘.
통학 버스의 장점과 중요성은 무엇인가? 

280
Dmitri Ivanovsky가 누구야?
Dmitri Ivanovsky의 정체 또는 업적 

10
피임을 하기 위한 방법중 약으로 처리하는 방법은 쓸만한가?
피임 약의 효과와 유용성은 어떠한가? 

100
헬륨이 다른 원소들과 반응을 잘 안하는 이유는?
헬륨이 화학적으로 비활성인 이유는 무엇인가? 

279
문맹 비율이 사회 발전에 미치는 영향은?
문맹 비율 사회 발전 영향 

42
이란 콘트라 사건이 무엇인지와 이 사건이 미국 정치에 미친 영향을 설명해 주세요.
이란 콘트라 사건이란 무엇이며 미국 정치에 미친 영향은 무엇인가? 

308
자기장이 얼마나 센지 표현하는 방식은?
자기장 세기를 표현하는 방법은 무엇인가? 

